In [ ]:
using Revise
using QuanticsTCI
using Test
using StaticArrays
using BenchmarkTools

In [ ]:
R = 10
d = 3
B = 4

# Grid on [1, 2, ..., d^R]
grid = QuanticsTCI.InherentDiscreteGrid{d,B}(R)

In [ ]:
f = x->1.0
qf(bitlist::AbstractVector{Int}) = f(QuanticsTCI.quantics_to_origcoord_fused(grid, bitlist))

In [ ]:
x = ones(Int, R)
@benchmark qf($x)

In [ ]:
#@benchmark QuanticsTCI.quantics_to_origcoord_fused(grid, $x)
@benchmark QuanticsTCI.quantics_to_index_fused(Val(B), Val(d), $x)

In [ ]:
qf2 = QuanticsTCI.quantics_function_fused(Float64, grid, f)

x = ones(Int, R)
@benchmark qf2($x)

In [ ]:
function testf(x, y, z)
    res = zeros(MVector{1, Float64})
    res .= x .+ y .+ z
    return res
end

a = (1,)
b = (1,)
c = (1,)

@benchmark testf($a, $b, $c)

In [ ]:
B = 3
R = 4
for index in 1:B^R
    bitlist = [mod(index, B^(R-i+1)) ÷ B^(R-i) + 1 for i in 1:R]
    @show bitlist
end

In [ ]:
x = ones(Int, 10)
y = ones(Int, 10)
z = ones(Int, 20)
xy = [x, y]
@benchmark QuanticsTCI.deinterleave_dimensions!(xy, z)

In [ ]:
n = 2
np = 2^n
for i1 in [1,2], i2 in 1:2
    q1 = index_to_quantics(i1, n)
    q2 = index_to_quantics(i2, n)
    @test fuse_dimensions(q1, q2) == index_to_quantics_fused([i1, i2], n)
    @test interleave_dimensions(q1, q2) == index_to_quantics_interleaved([i1, i2], n)
    @show i1
    @show q1
    @show i2
    @show q2
    @show fuse_dimensions(q1, q2)
end

In [ ]:
n = 2
i1, i2 = 1, 1
q1 = index_to_quantics(i1, n)
q2 = index_to_quantics(i2, n)
@benchmark fuse_dimensions($q1, $q2)

In [ ]:
result = ones(Int, n)
@benchmark QuanticsTCI.fuse_dimensions!($result, $q1, $q2)

In [ ]:
n = 10
i1, i2 = 1, 1
q1 = index_to_quantics(i1, n)
q2 = index_to_quantics(i2, n)
q12 = fuse_dimensions(q1, q2)

In [ ]:
@benchmark split_dimensions(Val(2), q12, 2)

In [ ]:
@test split_dimensions([1, 1, 1, 1], 1) == [[1, 1, 1, 1]]
@test split_dimensions([1, 1, 1, 1], 3) == [[1, 1, 1, 1] for i in 1:3]
@test split_dimensions([2, 2, 2, 2], 3) == [[2, 2, 2, 2], [1, 1, 1, 1], [1, 1, 1, 1]]
@test split_dimensions([3, 3, 3, 3], 3) == [[1, 1, 1, 1], [2, 2, 2, 2], [1, 1, 1, 1]]
@test split_dimensions([5, 5, 5, 5], 3) == [[1, 1, 1, 1], [1, 1, 1, 1], [2, 2, 2, 2]]
@test split_dimensions([1, 2, 4, 8], 3) == [[1, 2, 2, 2], [1, 1, 2, 2], [1, 1, 1, 2]]

In [ ]:
result = split_dimensions([1, 1, 1, 1], 3)
@show result
bitlist = [1, 1, 1, 1]
@benchmark QuanticsTCI.split_dimensions!(Val(2), result, bitlist)

In [ ]:
result